In [1]:
import pandas as pd
import xarray as xr

# Load sharks data

In [2]:
ss = pd.read_csv('AustraliaShark.csv')

# Load hydrodynamic data from eReefs

In [3]:
urls = 'https://thredds.ereefs.aims.gov.au/thredds/dodsC/ereefs/GBR4_H2p0_B3p1_Cq3b_Dhnd/monthly-monthly/EREEFS_AIMS-CSIRO_GBR4_H2p0_B3p1_Cq3b_Dhnd_bgc_monthly-monthly'
months = [11, 12, 1]
files = [f'{urls}-{year}-{month:02d}.nc' for year in range (2010, 2020) for month in months]

In [4]:
files1 = files [3:-3]
files1.append('https://thredds.ereefs.aims.gov.au/thredds/dodsC/ereefs/GBR4_H2p0_B3p1_Cq3b_Dhnd/monthly-monthly/EREEFS_AIMS-CSIRO_GBR4_H2p0_B3p1_Cq3b_Dhnd_bgc_monthly-monthly-2019-01.nc')

In [5]:
envVar = xr.open_mfdataset(files1, parallel = True)

Select hydrodynamic variables of interest

In [6]:
conVars = ['temp', 'salt', 'Chl_a_sum']
top = envVar.isel(k = slice (14,17))[conVars].mean(dim = 'k')
meandata = top.mean(dim = 'time')[conVars]

In [7]:
len(top['time'])

25

In [8]:
meandata[conVars]

<xarray.Dataset>
Dimensions:    (latitude: 723, longitude: 491)
Coordinates:
  * latitude   (latitude) float64 -28.7 -28.67 -28.64 ... -7.096 -7.066 -7.036
  * longitude  (longitude) float64 142.2 142.2 142.2 142.3 ... 156.8 156.8 156.9
Data variables:
    temp       (latitude, longitude) float32 dask.array<chunksize=(723, 491), meta=np.ndarray>
    salt       (latitude, longitude) float32 dask.array<chunksize=(723, 491), meta=np.ndarray>
    Chl_a_sum  (latitude, longitude) float32 dask.array<chunksize=(723, 491), meta=np.ndarray>

In [9]:
top

<xarray.Dataset>
Dimensions:    (time: 25, latitude: 723, longitude: 491)
Coordinates:
  * time       (time) datetime64[ns] 2010-12-31T14:00:00 ... 2018-12-31T14:00:00
  * latitude   (latitude) float64 -28.7 -28.67 -28.64 ... -7.096 -7.066 -7.036
  * longitude  (longitude) float64 142.2 142.2 142.2 142.3 ... 156.8 156.8 156.9
Data variables:
    temp       (time, latitude, longitude) float32 dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    salt       (time, latitude, longitude) float32 dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    Chl_a_sum  (time, latitude, longitude) float32 dask.array<chunksize=(1, 723, 491), meta=np.ndarray>

Clip the shark dataset to the extent of the eReefs region

In [10]:
lon_min, lon_max = float(meandata.longitude.min()), float(meandata.longitude.max())
lat_min, lat_max = float(meandata.latitude.min()), float(meandata.latitude.max())
ss['Longitude'] = pd.to_numeric(ss['Longitude'], errors = 'coerce')
ss['Latitude'] = pd.to_numeric(ss['Latitude'], errors = 'coerce')

In [11]:
ss_within_range = ss[(ss['Longitude'] >= lon_min) & (ss['Longitude'] <= lon_max) & (ss['Latitude'] >= lat_min) & (ss['Latitude'] <= lat_max)]

In [12]:
sharklats = ss_within_range['Latitude'].values
sharklons = ss_within_range['Longitude'].values

In [13]:
meandata.load()

<xarray.Dataset>
Dimensions:    (latitude: 723, longitude: 491)
Coordinates:
  * latitude   (latitude) float64 -28.7 -28.67 -28.64 ... -7.096 -7.066 -7.036
  * longitude  (longitude) float64 142.2 142.2 142.2 142.3 ... 156.8 156.8 156.9
Data variables:
    temp       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    salt       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    Chl_a_sum  (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan

# Extract 10 years average

In [14]:
chlPts = []
tempPts = []
saltPts = []
for k in range(len(sharklons)):
    tmp = meandata.sel(latitude=sharklats[k],longitude=sharklons[k],method='nearest')
    chlPts.append(tmp.Chl_a_sum.values+0.)
    tempPts.append(tmp.temp.values+0.)
    saltPts.append(tmp.salt.values+0.)

In [15]:
ss_within_range = ss_within_range.reset_index()
ss_within_range['salt_10years'] = saltPts
ss_within_range['temp_10years'] = tempPts
ss_within_range['Chl_a_sum_10years'] = chlPts

In [16]:
ss_within_range

,index,UIN,Incident.month,Incident.year,Victim.injury,State,Location,Latitude,Longitude,Site.category,...,Weather.condition,Air.temperature.°C,Personal.protective.device,Deterrent.brand.and.type,Data.source,Reference,Unnamed: 59,salt_10years,temp_10years,Chl_a_sum_10years
0,9,10.0,8.0,1836.0,fatal,QLD,fraser island,-25.483333,152.983333,coastal,...,NaN,NaN,NaN,NaN,book,"shipwrecks, evan mchugh",NaN,34.968910,27.702122,0.336258
1,10,11.0,8.0,1836.0,fatal,QLD,fraser island,-25.483333,152.983333,coastal,...,NaN,NaN,NaN,NaN,book,"shipwrecks, evan mchugh",NaN,34.968910,27.702122,0.336258
2,15,16.0,3.0,1847.0,injured,QLD,moreton bay,-27.250000,153.250000,coastal,...,NaN,NaN,NaN,NaN,media outlet,"green, whitley 1958, the queenslander 12.3.1921",NaN,35.267101,26.397655,0.189246
3,16,17.0,11.0,1847.0,injured,QLD,brisbane river,-27.433333,153.083333,river,...,NaN,NaN,NaN,NaN,media outlet,"morton bay courrier 4.12.1847, maitland mercur...",NaN,NaN,NaN,NaN
4,20,21.0,1.0,1850.0,fatal,QLD,moreton bay,-27.283333,153.216667,coastal,...,NaN,NaN,NaN,NaN,media outlet,the queenslander 6.2.1909,NaN,35.194664,26.557354,0.196261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,1156,1157.0,10.0,2020.0,injured,QLD,"britomart reef, hinchinbrook island",-18.250000,146.666944,island open ocean,...,NaN,NaN,NaN,NaN,NaN,https://www.theaustralian.com.au/breaking-news...,NaN,35.310478,28.412798,0.167240
383,1159,1160.0,11.0,2020.0,uninjured,QLD,"magpie reef, princess charlotte bay, cape york",-13.733300,143.866700,island open ocean,...,NaN,NaN,NaN,NaN,NaN,https://www.9news.com.au/national/divers-close...,NaN,35.412788,28.827227,0.095981
384,1182,1183.0,10.0,2021.0,injured,QLD,"hook passage, whitsundays",-20.266500,148.881900,island open ocean,...,NaN,NaN,NaN,NaN,NaN,https://7news.com.au/news/qld/whitsundays-shar...,NaN,35.417397,27.860708,0.295756
385,1187,1188.0,12.0,2021.0,injured,QLD,"rocky island, yarrabah, far north queensland",-16.906200,145.866200,island open ocean,...,NaN,NaN,NaN,NaN,NaN,https://www.dailymail.co.uk/news/article-10306...,NaN,35.094868,29.339853,0.188962


# Extract specific year dataset

In [20]:
year2010 = top.sel(time=slice('2010-01-01', '2011-01-01')).mean(dim='time')
year2010.load()

<xarray.Dataset>
Dimensions:    (latitude: 723, longitude: 491)
Coordinates:
  * latitude   (latitude) float64 -28.7 -28.67 -28.64 ... -7.096 -7.066 -7.036
  * longitude  (longitude) float64 142.2 142.2 142.2 142.3 ... 156.8 156.8 156.9
Data variables:
    temp       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    salt       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    Chl_a_sum  (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan

In [21]:
year2010

<xarray.Dataset>
Dimensions:    (latitude: 723, longitude: 491)
Coordinates:
  * latitude   (latitude) float64 -28.7 -28.67 -28.64 ... -7.096 -7.066 -7.036
  * longitude  (longitude) float64 142.2 142.2 142.2 142.3 ... 156.8 156.8 156.9
Data variables:
    temp       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    salt       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    Chl_a_sum  (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan

In [22]:
chlPts  = []
tempPts = []
saltPts = []
for k in range(len(sharklons)):
    tmp = year2010.sel(latitude=sharklats[k],longitude=sharklons[k],method='nearest')
    chlPts.append(tmp.Chl_a_sum.values+0.)
    tempPts.append(tmp.temp.values+0.)
    saltPts.append(tmp.salt.values+0.)

In [23]:
ss_within_range['salt_2010'] = saltPts
ss_within_range['temp_2010'] = tempPts
ss_within_range['Chl_a_sum_2010'] = chlPts

In [24]:
ss_within_range.to_csv('Aus_Shark_Attack_2010.csv',index=False)

Loop over all the years and extract the relevant information

In [25]:
for year in range(2011,2018):
    
    # Load the specific year
    tmpyear = top.sel(time=slice(str(year)+'-01-01', str(year+1)+'-01-01')).mean(dim='time')
    # tmpyear.load()
    
    # Extract data from shark attack
    chlPts = []
    tempPts = []
    saltPts = []
    for k in range(len(sharklons)):
        tmp = year2010.sel(latitude=sharklats[k],longitude=sharklons[k],method='nearest')
        chlPts.append(tmp.Chl_a_sum.values+0.)
        tempPts.append(tmp.temp.values+0.)
        saltPts.append(tmp.salt.values+0.)
    
    # Save it to the dataframe
    ss_within_range['salt_'+str(year)] = saltPts
    ss_within_range['temp_'+str(year)] = tempPts
    ss_within_range['Chl_a_sum_'+str(year)] = chlPts

In [26]:
ss_within_range.to_csv('Aus_Shark_Attack_2010.csv',index=False)

In [27]:
year2011 = top.sel(time=slice('2011-01-01', '2012-01-01')).mean(dim='time')
year2011.load()

<xarray.Dataset>
Dimensions:    (latitude: 723, longitude: 491)
Coordinates:
  * latitude   (latitude) float64 -28.7 -28.67 -28.64 ... -7.096 -7.066 -7.036
  * longitude  (longitude) float64 142.2 142.2 142.2 142.3 ... 156.8 156.8 156.9
Data variables:
    temp       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    salt       (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan
    Chl_a_sum  (latitude, longitude) float32 nan nan nan nan ... nan nan nan nan

Save to file

In [28]:
ss_within_range.to_csv('Aus_Shark_Attack.csv',index=False)

In [29]:
for year in range(2011,2018):
    
    # Load the specific year
    tmpyear = top.sel(time=slice(str(year)+'-01-01', str(year+1)+'-01-01')).mean(dim='time')
    tmpyear.load()
    
    # Extract data from shark attack
    chlPts = []
    tempPts = []
    saltPts = []
    for k in range(len(sharklons)):
        tmp = year2010.sel(latitude=sharklats[k],longitude=sharklons[k],method='nearest')
        chlPts.append(tmp.Chl_a_sum.values+0.)
        tempPts.append(tmp.temp.values+0.)
        saltPts.append(tmp.salt.values+0.)
    
    # Save it to the dataframe
    ss_within_range['salt_'+str(year)] = saltPts
    ss_within_range['temp_'+str(year)] = tempPts
    ss_within_range['Chl_a_sum_'+str(year)] = chlPts

In [41]:
df = pd.read_csv("Aus_Shark_Attack_2010.csv")

In [42]:
df.columns

Index(['index', 'UIN', 'Incident.month', 'Incident.year', 'Victim.injury',
       'State', 'Location', 'Latitude', 'Longitude', 'Site.category',
       'Site.category.comment', 'Shark.common.name', 'Shark.scientific.name',
       'Shark.identification.method', 'Shark.identification.source',
       'Shark.length.m', 'Basis.for.length', 'Provoked/unprovoked',
       'Provocative.act', 'No.sharks', 'Victim.activity', 'Fish.speared?',
       'Commercial.dive.activity', 'Object.of.bite', 'Present.at.time.of.bite',
       'Direction.first.strike', 'Shark.behaviour', 'Victim.aware.of.shark',
       'Shark.captured', 'Injury.location', 'Injury.severity', 'Victim.gender',
       'Victim.age', 'Victim.clothing', 'Clothing.coverage',
       'Dominant.clothing.colour', 'Other.clothing.colour', 'Clothing.pattern',
       'Fin.colour', 'Diversionary.action.taken',
       'Diversionary.action.outcome', 'People <3m', 'People 3-15m',
       'Time.of.incident', 'Depth.of.incident.m', 'Teeth.recovered',


In [44]:
import file_path

ModuleNotFoundError: No module named 'file_path'